# 0. MÓDULOS USADOS

In [1]:
use strict;
use warnings;
use Data::Dump qw(dump);
use d2l;
use Chart::Plotly::Trace::Bar;
use Chart::Plotly::Plot;

IPerl->load_plugin('Chart::Plotly');

# 1. LECTURA Y PREPROCESAMIENTO PARA LA ALINEACION DE LOS CORPUS

## 1.1 Lectura de archivos
Con la función leer_archivo se leerá cada uno de los corpus (OCR y REF) para el desarrollo del proyecto.
El uso de shift es para recibir parámetros en una función para extraer y eliminar el primer elemento de la lista de argumentos (necesario cuando se procesa parámetros de manera secuencial, obteniendolos uno por uno)
Encoding(UTF-8) sirve para garantizar que los caracteres se lean correctamente si el archivo contiene caracteres no ASCII.
Retorna un map que devuelve una nueva lista con los cambios pedidos por los regex.
1. Regex elimina espacios en blanco
2. Luego se aplica un regex para cambiar los caracteres no alfanuméricos por espacios en blanco con el fin de formar palabras solo con caracteres alfanuméricos
3. Regex que elimina espacios al principio y al final de cada linea
4. Conversión de caracteres a minúsculas
5. grep filtra lineas vacias o que contengan solo espacios en blanco

### 1.1.1 Lectura data original

In [3]:
sub leer_archivo {
    my $archivo = shift;

    open my $fh, '<:encoding(UTF-8)', $archivo or die "ERROR: no se pudo abrir el archivo '$archivo'\n";
    my @lineas;
    
    while (my $linea = <$fh>) {
        chomp $linea;
        $linea =~ s/\s//g;
        $linea =~ s/^\W+|\W+$|\W+/ /g;
        $linea =~ s/^\s+|\s+$//g;
        push @lineas, lc($linea) unless $linea =~ /^\s+$/;
    }
    close $fh;
    
    return \@lineas;
}

## 1.2 Asignación de variables con el contenido de cada corpus ya preprocesado

In [4]:
my $tiempo = new d2l::Timer();

my $corpus_OCR = leer_archivo("../../TesisModelo1/0_Corpus/0_Entrenamiento/train.tok.fr");
my $corpus_REF = leer_archivo("../../TesisModelo1/0_Corpus/0_Entrenamiento/train.tok.en");

print "Duracion: ", $tiempo->stop(), "s.\n";

Duracion: 21.1546559333801s.


1

## 1.3 Revisión y segundo preprocesamiento de los corpus

### 1.3.1 Comprobación de la cadena más larga en los dos corpus

In [5]:
sub oracion_mas_larga {
  my $corpus = shift;

  my $oracion_mas_larga = "";
  my $longitud_maxima = 0;

  foreach my $oracion (@{$corpus}) {
    my @palabras = split(' ', $oracion);
    my $longitud_oracion = scalar @palabras;

    if ($longitud_oracion > $longitud_maxima) {
      $longitud_maxima = $longitud_oracion;
      $oracion_mas_larga = $oracion;
    }
  }

  return $oracion_mas_larga, $longitud_maxima;
}

In [6]:
my ($oracion_mas_larga_OCR, $longitud_mas_larga_OCR) = oracion_mas_larga($corpus_OCR);
my ($oracion_mas_larga_REF, $longitud_mas_larga_REF) = oracion_mas_larga($corpus_REF);

print "La oración más larga en el archivo OCR es:\n\n$oracion_mas_larga_OCR\n\n";
print "Número de palabras: $longitud_mas_larga_OCR\n\n\n";

print "La oración más larga en el archivo REF es:\n\n$oracion_mas_larga_REF\n\n";
print "Número de palabras: $longitud_mas_larga_REF\n\n";

La oración más larga en el archivo OCR es:

assuming there are two msts called t and t for any edge e in t if we refove e from t then t becomes unconnected and we have a cut s v s according to exercise 23 1 3 e is the light edge through cut s v s if edge x is in t and through cut s v s then x is also a light weight because the light edge is unique so e and x is the same edge e is also in t because we choose e at random of all edges in t also in t as a result the mst is unique

Número de palabras: 108


La oración más larga en el archivo REF es:

assuming there are two msts called t and t for any edge e in t if we remove e from t then t becomes unconnected and we have a cut s v s according to exercise 23 1 3 e is the light edge through cut s v s if edge x is in t and through cut s v s then x is also a light weight because the light edge is unique so e and x is the same edge e is also in t because we choose e at random of all edges in t also in t as a result the mst is unique

Número de 

1

### 1.3.2 Cálculo de la frecuencia de palabras por fila

In [7]:
sub frecuencia_palabras_por_fila {
    my ($corpus) = @_;
    my %contador_frecuencia_palabras;
    undef %contador_frecuencia_palabras;

    foreach my $oracion (@{$corpus}) {
        my @palabras = split(' ', $oracion);
        my $oracion_len = scalar @palabras;

        $contador_frecuencia_palabras{$oracion_len}++;
    }

    return %contador_frecuencia_palabras;
}

### 1.3.3 Histograma de frecuencias y número de palabras

In [8]:
sub histograma {
    my ($titulo, $len, $frecuencias, $nombre) = @_;

    my $bar_trace = Chart::Plotly::Trace::Bar->new(
        x    => $len,
        y    => $frecuencias,
        name => $nombre,
    );

    my $plot = Chart::Plotly::Plot->new(
        traces => [$bar_trace],
        layout => {
            title => {
                text => $titulo,
                font => {
                    family => "Arial",
                    size   => 30
                }
            },
            xaxis => { title => 'Número de palabras' },
            yaxis => { title => 'Frecuencia' },
        },
    );

    return $plot;
}

In [9]:
my %contador_frecuencia_palabras_REF = frecuencia_palabras_por_fila($corpus_REF);
my %contador_frecuencia_palabras_OCR = frecuencia_palabras_por_fila($corpus_OCR);

my @len_REF = sort { $a <=> $b } keys %contador_frecuencia_palabras_REF;
my @frecuencias_REF = @contador_frecuencia_palabras_REF{@len_REF};

my @len_OCR = sort { $a <=> $b } keys %contador_frecuencia_palabras_OCR;
my @frecuencias_OCR = @contador_frecuencia_palabras_OCR{@len_OCR};

2584776108181171010677962094929589106401240414057158991583214514115639002675354904406391232942999252123751869178216601568137214211210118812461186108010191003102093084077276595077479074774080168856357959458648650952246943740943846942533436037037427327229223928626832127220322618115817016214313012214173119804730322623121513410453111

In [10]:
my $plot_OCR = histograma(
    'Distribución de frecuencias de número de palabras de OCR',
    \@len_OCR,
    \@frecuencias_OCR,
    'Frecuencia OCR'
);

In [11]:
my $plot_REF = histograma(
    'Distribución de frecuencias de número de palabras de REF',
    \@len_REF,
    \@frecuencias_REF,
    'Frecuencia REF'
);

### 1.3.4 Comprobación de filas vacías, filas que esten vacias en un archivo y que en el otro tenga una sola palabra, filas en la misma posicion de los dos archivos que solo contengan una palabra y caracteres individuales en los archivos

In [12]:
sub numero_palabras {
    my ($corpus, $target_len) = @_;
    my $pos = 1;
    my $filas_encontradas = 0;

    foreach my $oracion (@{$corpus}) {
        my @palabras = split(' ', $oracion);
        my $oracion_len = scalar @palabras;

        if ($oracion_len == $target_len) {
            print "Fila con $target_len tokens encontrada en la posición $pos\n";
            $filas_encontradas++;
        }

        $pos++;
    }

    return $filas_encontradas;
}

In [13]:
my $target_len_OCR = 1;
my $filas_encontradas_OCR = numero_palabras($corpus_OCR, $target_len_OCR);
print "Número de filas encontradas en OCR: $filas_encontradas_OCR\n";

Fila con 1 tokens encontrada en la posición 6
Fila con 1 tokens encontrada en la posición 10
Fila con 1 tokens encontrada en la posición 49
Fila con 1 tokens encontrada en la posición 58
Fila con 1 tokens encontrada en la posición 60
Fila con 1 tokens encontrada en la posición 81
Fila con 1 tokens encontrada en la posición 82
Fila con 1 tokens encontrada en la posición 83
Fila con 1 tokens encontrada en la posición 161
Fila con 1 tokens encontrada en la posición 176
Fila con 1 tokens encontrada en la posición 177
Fila con 1 tokens encontrada en la posición 445
Fila con 1 tokens encontrada en la posición 500
Fila con 1 tokens encontrada en la posición 643
Fila con 1 tokens encontrada en la posición 687
Fila con 1 tokens encontrada en la posición 703
Fila con 1 tokens encontrada en la posición 761
Fila con 1 tokens encontrada en la posición 762
Fila con 1 tokens encontrada en la posición 763
Fila con 1 tokens encontrada en la posición 769
Fila con 1 tokens encontrada en la posición 845
F

1

In [14]:
my $target_len_REF = 1;
my $filas_encontradas_REF = numero_palabras($corpus_REF, $target_len_REF);
print "Número de filas encontradas en REF: $filas_encontradas_REF\n";

Fila con 1 tokens encontrada en la posición 6
Fila con 1 tokens encontrada en la posición 10
Fila con 1 tokens encontrada en la posición 49
Fila con 1 tokens encontrada en la posición 58
Fila con 1 tokens encontrada en la posición 60
Fila con 1 tokens encontrada en la posición 161
Fila con 1 tokens encontrada en la posición 176
Fila con 1 tokens encontrada en la posición 177
Fila con 1 tokens encontrada en la posición 445
Fila con 1 tokens encontrada en la posición 500
Fila con 1 tokens encontrada en la posición 643
Fila con 1 tokens encontrada en la posición 687
Fila con 1 tokens encontrada en la posición 703
Fila con 1 tokens encontrada en la posición 769
Fila con 1 tokens encontrada en la posición 845
Fila con 1 tokens encontrada en la posición 899
Fila con 1 tokens encontrada en la posición 990
Fila con 1 tokens encontrada en la posición 1056
Fila con 1 tokens encontrada en la posición 1077
Fila con 1 tokens encontrada en la posición 1256
Fila con 1 tokens encontrada en la posición

1

In [15]:
sub caracteres_unicos {
    my ($corpus) = @_;

    my @matching_lineas;

    for (my $i = 0; $i < scalar @$corpus; $i++) {
        my $linea = $corpus->[$i];
        if ($linea =~ /^(\w\s\w\s?\w\s?)+\w$/) {
            push @matching_lineas, { linea_numero => $i + 1, linea_contenido => $linea };
        }
    }

    return @matching_lineas;
}

In [16]:
my @caracteres_unicos_OCR = caracteres_unicos($corpus_OCR);

foreach my $linea (@caracteres_unicos_OCR) {
    my $linea_numero = $linea->{linea_numero};
    my $linea_contenido = $linea->{linea_contenido};
    print "Se encontró una fila con caracteres individuales separados por espacios en la fila $linea_numero: $linea_contenido\n";
}

Se encontró una fila con caracteres individuales separados por espacios en la fila 22577: 3 3 go web
Se encontró una fila con caracteres individuales separados por espacios en la fila 22584: 3 3 go web
Se encontró una fila con caracteres individuales separados por espacios en la fila 31275: 0 5 0 0 0 75
Se encontró una fila con caracteres individuales separados por espacios en la fila 32562: 2 11 7
Se encontró una fila con caracteres individuales separados por espacios en la fila 32567: 2 11 7
Se encontró una fila con caracteres individuales separados por espacios en la fila 59821: f php
Se encontró una fila con caracteres individuales separados por espacios en la fila 59949: f php
Se encontró una fila con caracteres individuales separados por espacios en la fila 60107: f php
Se encontró una fila con caracteres individuales separados por espacios en la fila 60425: f php
Se encontró una fila con caracteres individuales separados por espacios en la fila 60745: f php
Se encontró una fila 

In [17]:
my @caracteres_unicos_REF = caracteres_unicos($corpus_REF);

foreach my $linea (@caracteres_unicos_REF) {
    my $linea_numero = $linea->{linea_numero};
    my $linea_contenido = $linea->{linea_contenido};
    print "Se encontró una fila con caracteres individuales separados por espacios en la fila $linea_numero: $linea_contenido\n";
}

Se encontró una fila con caracteres individuales separados por espacios en la fila 22577: 3 3 go web
Se encontró una fila con caracteres individuales separados por espacios en la fila 22584: 3 3 go web
Se encontró una fila con caracteres individuales separados por espacios en la fila 31275: 0 5 0 0 0 75
Se encontró una fila con caracteres individuales separados por espacios en la fila 32562: 2 11 7
Se encontró una fila con caracteres individuales separados por espacios en la fila 32567: 2 11 7
Se encontró una fila con caracteres individuales separados por espacios en la fila 71591: 4 5 8 9
Se encontró una fila con caracteres individuales separados por espacios en la fila 73206: o i a i j x j
Se encontró una fila con caracteres individuales separados por espacios en la fila 73208: o i a i j x j
Se encontró una fila con caracteres individuales separados por espacios en la fila 73927: m ppc shr srw
Se encontró una fila con caracteres individuales separados por espacios en la fila 101105: 

### 1.3.4 Eliminación de filas vacias

In [18]:
sub comparar_filtrar {
    my ($corpus_OCR, $corpus_REF) = @_;

    my @corpus_OCR = @{$corpus_OCR};
    my @corpus_REF = @{$corpus_REF};

    my @filtrar_corpus_OCR;
    my @filtrar_corpus_REF;

    for my $i (0 .. $#corpus_OCR) {
        my $linea_OCR = $corpus_OCR[$i];
        my $linea_REF = $corpus_REF[$i];

        chomp $linea_OCR;
        chomp $linea_REF;

        next if $linea_OCR =~ /^(\w\s\w\s?\w\s?)+\w$/ || $linea_REF =~ /^(\w\s\w\s?\w\s?)+\w$/;
        next if $linea_OCR =~ /^\s*$/ && $linea_REF =~ /^\s*$/;
        next if $linea_OCR =~ /^\w+$/ && $linea_REF =~ /^\w+$/;

        if ($linea_OCR =~ /^\s*$/ && $linea_REF =~ /^\w+$/) {
            next;
        }

        if ($linea_OCR =~ /^\w+$/ && $linea_REF =~ /^\s*$/) {
            next;
        }

        push @filtrar_corpus_OCR, $linea_OCR;
        push @filtrar_corpus_REF, $linea_REF;
    }

    return (\@filtrar_corpus_OCR, \@filtrar_corpus_REF);
}

In [19]:
($corpus_OCR, $corpus_REF) = comparar_filtrar($corpus_OCR, $corpus_REF);

ARRAY(0x12325028)ARRAY(0x12324f80)

### 1.3.5 Eliminar oraciones que tengan una sola palabra

In [20]:
sub contador_palabras {
    my ($corpus_OCR, $corpus_REF) = @_;

    my @filtrar_OCR;
    my @filtrar_REF;

    for (my $i = 0; $i < scalar @$corpus_OCR; $i++) {
        my $linea_OCR = $corpus_OCR->[$i];
        my $linea_REF = $corpus_REF->[$i];

        my @palabras_OCR = split /\s+/, $linea_OCR;
        my @palabras_REF = split /\s+/, $linea_REF;

        if ((scalar @palabras_OCR<2 || scalar @palabras_REF<2)) {
            next;
        }

        push @filtrar_OCR, $linea_OCR;
        push @filtrar_REF, $linea_REF;
    }

    return (\@filtrar_OCR, \@filtrar_REF);
}


In [21]:
($corpus_OCR, $corpus_REF) = contador_palabras($corpus_OCR, $corpus_REF);

ARRAY(0xde13480)ARRAY(0xde13498)

In [22]:
my %contador_frecuencia_palabras_REF = frecuencia_palabras_por_fila($corpus_REF);
my %contador_frecuencia_palabras_OCR = frecuencia_palabras_por_fila($corpus_OCR);

my @len_REF = sort { $a <=> $b } keys %contador_frecuencia_palabras_REF;
my @frecuencias_REF = @contador_frecuencia_palabras_REF{@len_REF};

my @len_OCR = sort { $a <=> $b } keys %contador_frecuencia_palabras_OCR;
my @frecuencias_OCR = @contador_frecuencia_palabras_OCR{@len_OCR};

106811169810663961994899586106401240414056158991583214512115639002675354904406391232942999252123751869178216601568137214211210118812461186108010191003102093084077276595077479074774080168856357959458648650952246943740943846942533436037037427327229223928626832127220322618115817016214313012214173119804730322623121513410453111

In [23]:
my $plot_OCR = histograma(
    'Distribución de frecuencias de número de palabras de OCR',
    \@len_OCR,
    \@frecuencias_OCR,
    'Frecuencia OCR'
);

In [24]:
my $plot_REF = histograma(
    'Distribución de frecuencias de número de palabras de REF',
    \@len_REF,
    \@frecuencias_REF,
    'Frecuencia REF'
);

In [25]:
# for my $let (0 .. 100) {
#     print dump($corpus_OCR->[$let])."\n";
# }

### 1.3.6 Contar el número de caracteres por palabra

In [26]:
sub contar_caracteres_por_palabra {
    my ($corpus, $top_n) = @_;
    my %palabra_info;

    for my $fila (0 .. $#{$corpus}) {
        my @palabras = split(" ", $corpus->[$fila]);
        for my $palabra (@palabras) {
            $palabra =~ s/^\s+|\s+$//g;  # Elimina espacios en blanco al principio y al final
            my $longitud = length($palabra);
            push @{$palabra_info{$palabra}}, [$longitud, $fila + 1];
        }
    }

    # Ordenar el resultado por longitud de caracteres en orden descendente
    my @resultado_ordenado = sort { length($b) <=> length($a) } keys %palabra_info;

    # Mostrar las palabras más largas
    for my $i (0 .. $top_n - 1) {
        last if $i >= @resultado_ordenado;
        my $palabra = $resultado_ordenado[$i];
        my $info = $palabra_info{$palabra};
        my $fila_maxima = (sort { $b->[0] <=> $a->[0] } @$info)[0];
        print "Palabra: $palabra, Longitud: $fila_maxima->[0], Fila: $fila_maxima->[1]\n";
    }
}

In [27]:
contar_caracteres_por_palabra($corpus_OCR, 10);

Palabra: enqtnzm0njmwotc3nti0ltblnmfhodnhymrkmwuxzmu4ztq2mdjinjlmywziodnjmdu5ogrjythlzmzjmtc5ymq3mzuzmdlhmzk0zdgzmdy, Longitud: 107, Fila: 52901
Palabra: enqtmza0nzg2otaznzq1ltuxotjly2yyntvlngq3mtjkotiwztu2yjy3yzrjzwfiyze4ode5odzinja4ngi5ntjhzmu2ztllotmwzdhjmza, Longitud: 107, Fila: 29467
Palabra: upgradinganmvc4visualstudio2010solutiontovisualstudio2012andnet45andef5, Longitud: 71, Fila: 21304
Palabra: bjrtbgmxywntnxr0nxq1a2hqzmrkntrncgnaz3jvdxauy2fszw5kyxiuz29vz2xllmnvbq, Longitud: 70, Fila: 222417
Palabra: itshouldresolvetoanobjectwiththepassphraseandthepublickeyandtheaddress, Longitud: 70, Fila: 123873
Palabra: 0x42073576a160206e61b4d9b70b436359b8d220f8b88c7c272c77023513c62c3d, Longitud: 66, Fila: 159130
Palabra: 10bd216cab733340447a2ef531d24284a319284cfa673fa71b908affefa55278, Longitud: 64, Fila: 38762
Palabra: 5drnk1whyb6cz27sifb87q3rf8wx3xittjxwqadtohqkals8cdyoiydj3gky9w, Longitud: 62, Fila: 195787
Palabra: insertspaceafteropeningandbeforeclosingnonemptyparenthesis, Longitud: 58

In [28]:
open(my $archivo_OCR, '>:encoding(UTF-8)', '../../TesisModelo1/0_Corpus/1_Entrenamiento/corpus_OCR_preprocesada.txt') or die "No se puede abrir el archivo corpus_OCR_preprocesada.txt: $!";
foreach my $linea (@{$corpus_OCR}) {
    print $archivo_OCR $linea . "\n";
}
close($archivo_OCR);

open(my $archivo_REF, '>:encoding(UTF-8)', '../../TesisModelo1/0_Corpus/1_Entrenamiento/corpus_REF_preprocesada.txt') or die "No se puede abrir el archivo corpus_REF_preprocesada.txt: $!";
foreach my $linea (@{$corpus_REF}) {
    print $archivo_REF $linea . "\n";
}
close($archivo_REF);

1